# Sample Code: CNNs for handwritten digit recognition

Example of image classification using *Convolutional Neural Net* with 2D convolutions.

Reference: https://nextjournal.com/gkoehler/pytorch-mnist

In [ ]:
%config InlineBackend.figure_format = 'svg'

In [ ]:
# GPU support
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using device:', device)

In [ ]:
import torch
import torchvision
import numpy as np
import os, time, random
from torch.utils.data import Dataset, DataLoader
from sklearn import metrics
from matplotlib import pyplot as plt
import itertools

In [ ]:
train_dataset = torchvision.datasets.MNIST(
    'mnist/', train=True, download=True,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,))
    ])
)
test_dataset = torchvision.datasets.MNIST(
    'mnist/', train=False, download=True,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,))
    ])
)

batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
examples = enumerate(train_loader)
batch_idx, (example_data, example_targets) = next(examples)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
for i in range(6):
    plt.subplot(2,3,i+1)
    plt.tight_layout()
    plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
    plt.title("Ground Truth: {}".format(example_targets[i]))
    plt.xticks([])
    plt.yticks([])
plt.show()

In [ ]:
train_dataset.data.shape

## CNN class

Convolution → Max Pooling → Convolution → Max Pooling → Flatten → Fully-connected layer → ReLU → Fully-connected layer → (Softmax + Cross-Entropy)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 8, 5, padding=2)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(8, 16, 5, padding=2)
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(int(28*28/2/2/2/2)*16, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        N, C, W, H = x.shape
        z = self.pool1(torch.relu(self.conv1(x)))
        z = self.pool2(torch.relu(self.conv2(z)))
        z = z.view(N, -1)
        z = F.relu(self.fc1(z))
        z = self.fc2(z)
        return z

## Train

In [ ]:
def train(network, epoch):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = network(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            train_losses.append(loss.item())
            train_counter.append((batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

def test(network):
    network.eval()
    test_loss = []
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = network(data)
            test_loss.append(criterion(output, target).item())
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss = np.mean(test_loss)
    test_losses.append(test_loss)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
n_epochs = 5
learning_rate = 1e-3
log_interval = 100


model = Net()
print('Number of float-valued parameters:', count_parameters(model))

model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

test(model)
for epoch in range(1, n_epochs + 1):
    train(model, epoch)
    test(model)

In [ ]:
fig = plt.figure()
plt.plot(train_counter, train_losses, color='blue')
plt.scatter(test_counter[1:], test_losses[1:], color='red')
plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
plt.xlabel('number of training examples seen')
plt.ylabel('cross entropy loss')
plt.show()